# Sutton 
 
## Chapter 3 - The Reinforcement Learning Problem

In [8]:
import random
import matplotlib.pyplot as plt
import numpy as np
import numpy.random 

Gridworld

Calculate $P^{\pi}(v)$ using dynamic programming 

In [72]:
class Grid:
    
    def __init__(self):
        """ Setup new game board. """
        self.grid = {}
        self.width = 5
        self.height = 5
        self.gamma = 0.9
        
        # setup grid
        for x in range(1,self.width+1):
            for y in range(1,self.height+1):
                if (x, y) == (2,1):
                    self.grid[(x,y)] = (10, 2, 5)
                elif (x, y) == (4,1):
                    self.grid[(x,y)] = (5, 4, 3)
                else:
                    self.grid[(x,y)] = (0, 0, 0)
              
    def get_new_position(self, x, y, dx, dy):
        """ Returns (reward, x, y) after a move from (x,y) in the direction (dx, dy) """

        (reward, teleportX, teleportY) = self.grid[(x, y)]
        
        if (teleportX != 0):
            return (reward, teleportX, teleportY)
        else:
            if (x + dx, y + dy) in self.grid:
                return (0, x + dx, y + dy)
            else:
                return (-1, x, y)    
    

def solve_dp(grid):
    """ Finds expected value at each square under random movement policy 
        using dynamic programming. """

    moves = [(-1,0),(+1,0),(0,-1),(0,+1)]

    NUM_STEPS = 100
    
    result = {}
    for x in range(1,grid.width+1):
        for y in range(1,grid.height+1):
            result[(x,y)] = 0.0
            
    # we solve the bellman equations using dp
    for step in range(NUM_STEPS):
        old_result = result.copy()
        for x in range(1,grid.width+1):
            for y in range(1,grid.height+1):
                result[(x,y)] = 0.0
                for (dx,dy) in moves:
                    (reward, nx, ny) = grid.get_new_position(x, y, dx, dy)
                    # bellman is pi(x,y) = (prob action occurs) * (reward for this state + discount * previous estimated pi(new state))
                    # note: this feels a lot like finding a stable solution to a markov chain...
                    result[(x,y)] += 0.25 * (reward + grid.gamma * old_result[(nx, ny)])
            
    return result
    


def solve_random_search(grid):
    """ Finds expected value at each square via a randomised search. """
    
    moves = [(-1,0),(+1,0),(0,-1),(0,+1)]
    
    NUM_TRIALS = 200
    NUM_MOVES = 100
    
    result = {}
    
    for x in range(1,grid.width+1):
        for y in range(1,grid.height+1):
            total_score = 0
            for i in range(NUM_TRIALS):
                score = 0
                atX = x
                atY = y
                for j in range(NUM_MOVES):
                    dx, dy = moves[random.randint(0,3)]
                    (reward, atX, atY) = grid.get_new_position(atX, atY, dx, dy)
                    score += (grid.gamma ** j) * reward
                total_score += score
            result[(x,y)] = total_score / NUM_TRIALS 
    
    return result
            
def print_result(result):
    
    for y in range(1,5+1):
        s = ""
        for x in range(1,5+1):
            s += "{0:.01f}".format(result[(x,y)]).ljust(8)
        print(s)
        
grid = Grid()

result = solve_random_search(grid)
print("Random search solution.")
print_result(result)

result = solve_dp(grid)
print("DP solution.")
print_result(result)

                

Random search solution.
3.5     8.7     4.4     5.2     1.2     
1.4     3.3     2.6     1.9     0.6     
0.2     0.8     0.7     0.4     -0.6    
-0.9    -0.4    -0.3    -0.7    -1.2    
-2.0    -1.3    -1.4    -1.6    -1.8    
DP solution.
3.3     8.8     4.4     5.3     1.5     
1.5     3.0     2.3     1.9     0.5     
0.1     0.7     0.7     0.4     -0.4    
-1.0    -0.4    -0.4    -0.6    -1.2    
-1.9    -1.3    -1.2    -1.4    -2.0    
